In [ ]:
import scipy.linalg
import scipy.optimize
import sys
import numpy as np
import math
import importlib
import matplotlib.pyplot as plt
import scripts.theodolite_function as tfu
tfu = importlib.reload(tfu)
from scipy.interpolate import splprep, splev
from scipy import interpolate
import seaborn as sns
from matplotlib.colors import BoundaryNorm
from matplotlib.ticker import MaxNLocator
from scipy.spatial.transform import Rotation as R
import scripts.theodolite_utils as ttfu


def resection_with_2_known_points(point_ref_1, point_ref_2, point_meas_1, point_meas_2):
    
    station_z = np.mean([(point_ref_1[2] - point_meas_1[2]*np.sin(point_meas_1[0])),(point_ref_2[2] - point_meas_2[2]*np.sin(point_meas_2[0]))])
    
    # find the position of the total station using the intersection of 2 circles given by the measurements of the prism position.
    radius_1 = point_meas_1[2]*np.cos(point_meas_1[0])#np.sqrt(point_meas_1[0]**2+point_meas_1[1]**2)
    radius_2 = point_meas_2[2]*np.cos(point_meas_2[0])#np.sqrt(point_meas_2[0]**2+point_meas_2[1]**2)
    
    delta_x = point_ref_2[0] - point_ref_1[0]
    delta_y = point_ref_2[1] - point_ref_1[1]
    
    # find the value of y based on the circles' radius and the reference points
    y = (radius_1**2 - radius_2**2 - point_ref_1[1]**2 + point_ref_2[1]**2)/(2*delta_y)
    
    # find the possible values of x based on the y value
    x_1_1 = np.sqrt(radius_1**2 - (y - point_ref_1[1])**2)
    x_2_1 = -np.sqrt(radius_1**2 - (y - point_ref_1[1])**2)   
    x_1_2 = np.sqrt(radius_2**2 - (y - point_ref_2[1])**2)
    x_2_2 = -np.sqrt(radius_2**2 - (y - point_ref_2[1])**2)

    station_pos_complete_1 = np.array([np.mean([x_1_1, x_1_2]), y, station_z, 0, 0, 0])   
    station_pos_complete_2 = np.array([np.mean([x_2_1, x_2_2]), y, station_z, 0, 0, 0]) 
    
    return [station_pos_complete_1, station_pos_complete_2]


def calculate_tf_angle_using_scalar_product(measured_points, reference_points, calib_index):
    angle = 0
    # we need a case for every calib_index, because each calibration uses 2 different pillars and thus the reference_line is specific to the calib_index.
    if calib_index==0:
        # make the vectors unit vectors
        measured_line = (measured_points[0][:3] - measured_points[1][:3])/np.linalg.norm(measured_points[0][:3] - measured_points[1][:3])
        reference_line = (reference_points[0] - reference_points[1])/np.linalg.norm(reference_points[0] - reference_points[1])
        angle = np.arccos((np.dot(reference_line, measured_line)/(np.linalg.norm(measured_line)*np.linalg.norm(reference_line))))
        if measured_points[0][0]>reference_points[0][0]:
            angle = -angle
    elif calib_index==1:
        # make the vectors unit vectors
        measured_line = (measured_points[0][:3] - measured_points[2][:3])/np.linalg.norm(measured_points[0][:3] - measured_points[2][:3])
        reference_line = (reference_points[0] - reference_points[2])/np.linalg.norm(reference_points[0] - reference_points[2])
        angle = np.arccos((np.dot(reference_line, measured_line)/(np.linalg.norm(measured_line)*np.linalg.norm(reference_line))))
        if measured_points[0][0]>reference_points[0][0]:
            angle = -angle
    elif calib_index==2:
        # make the vectors unit vectors
        measured_line = (measured_points[0][:3] - measured_points[3][:3])/np.linalg.norm(measured_points[0][:3] - measured_points[3][:3])
        reference_line = (reference_points[0] - reference_points[3])/np.linalg.norm(reference_points[0] - reference_points[3])
        angle = np.arccos((np.dot(reference_line, measured_line)/(np.linalg.norm(measured_line)*np.linalg.norm(reference_line))))
        if measured_points[0][0]>reference_points[0][0]:
            angle = -angle
    elif calib_index==3:
        # make the vectors unit vectors
        measured_line = (measured_points[1][:3] - measured_points[2][:3])/np.linalg.norm(measured_points[1][:3] - measured_points[2][:3])
        reference_line = (reference_points[1] - reference_points[2])/np.linalg.norm(reference_points[1] - reference_points[2])
        angle = np.arccos((np.dot(reference_line, measured_line)/(np.linalg.norm(measured_line)*np.linalg.norm(reference_line))))
        if measured_points[1][0]>reference_points[1][0]:
            angle = -angle
    elif calib_index==4:
        measured_line = (measured_points[1][:3] - measured_points[3][:3])/np.linalg.norm(measured_points[1][:3] - measured_points[3][:3])
        reference_line = (reference_points[1] - reference_points[3])/np.linalg.norm(reference_points[1] - reference_points[3])
        angle = np.arccos((np.dot(reference_line, measured_line)/(np.linalg.norm(measured_line)*np.linalg.norm(reference_line))))
        if measured_points[1][0]>reference_points[1][0]:
            angle = -angle
    elif calib_index==5:
        # make the vectors unit vectors
        measured_line = (measured_points[3][:3] - measured_points[2][:3])/np.linalg.norm(measured_points[3][:3] - measured_points[2][:3])
        reference_line = (reference_points[3] - reference_points[2])/np.linalg.norm(reference_points[3] - reference_points[2])
        angle = np.arccos((np.dot(reference_line, measured_line)/(np.linalg.norm(measured_line)*np.linalg.norm(reference_line))))
        if measured_points[3][0]<reference_points[3][0]:
            angle = -angle
    T = np.identity(4)
    Rot_r = np.array([[np.cos(angle), -np.sin(angle), 0],
                 [np.sin(angle), np.cos(angle), 0],
                 [0, 0, 1]])
    T[0:3,0:3]=Rot_r
    # print(T)
    return T

In [ ]:
%matplotlib inline

pilier_A = np.array([0, 0, 0])
pilier_B = np.array([0, np.sqrt(181.016**2 - 0.323**2), 0.323])
pilier_C = np.array([0, np.sqrt(363.492**2 - 1.038**2), 1.038])
pilier_D = np.array([0, np.sqrt(548.578**2 - 2.374**2), 2.374])

#pilier_A = np.array([0, 0, 0])
#pilier_B = np.array([np.sqrt(181.016**2 - 0.323**2), 0, 0.323])
#pilier_C = np.array([np.sqrt(363.492**2 - 1.038**2), 0, 1.038])
#pilier_D = np.array([np.sqrt(548.578**2 - 2.374**2), 0, 2.374])

liste_piliers = [pilier_A, pilier_B, pilier_C, pilier_D]

# Read markers file of grand axe and show them
tfu = importlib.reload(tfu)
ttfu = importlib.reload(ttfu)

file_name = './data/total_station/20220711/theodolite_reference_prisms_geomatic.txt'

raw_1, raw_2, raw_3, trimble_1, trimble_2, trimble_3, T_1_grand, T_2_grand, T_3_grand = ttfu.read_marker_file_raw_data(file_name)
station_1_pos1 = []
station_1_pos2 = []
station_2_pos1 = []
station_2_pos2 = []
station_3_pos1 = []
station_3_pos2 = []

# compute all total station poses possible for the 6 different pillar combination.
for i in range(len(liste_piliers)):
    for j in range(i+1, len(liste_piliers)):
        print(i,j)
        possible_poses_station_1 = resection_with_2_known_points(liste_piliers[i], liste_piliers[j], raw_1[i], raw_1[j])
        for pose, station in zip(possible_poses_station_1, [station_1_pos1, station_1_pos2]):
            station.append(pose)
        possible_poses_station_2 = resection_with_2_known_points(liste_piliers[i], liste_piliers[j], raw_2[i], raw_2[j])
        for pose, station in zip(possible_poses_station_2, [station_2_pos1, station_2_pos2]):
            station.append(pose)
        possible_poses_station_3 = resection_with_2_known_points(liste_piliers[i], liste_piliers[j], raw_3[i], raw_3[j])
        for pose, station in zip(possible_poses_station_3, [station_3_pos1, station_3_pos2]):
            station.append(pose)

#station_1_pos1, station_1_pos2 = resection_with_2_known_points(pilier_D, pilier_C, raw_1[0], raw_1[1])
#station_2_pos1, station_2_pos2 = resection_with_2_known_points(pilier_D, pilier_C, raw_2[0], raw_2[1])
#station_3_pos1, station_3_pos2 = resection_with_2_known_points(pilier_D, pilier_C, raw_3[0], raw_3[1])
# print("Sation 1:")
# print("possible position 1: ", station_1_pos1)
# print("possible position 2: ", station_1_pos2)
# print("Station 2:")
# print("possible position 1: ", station_2_pos1)
# print("possible position 2: ", station_2_pos2)
# print("Station 3:")
# print("possible position 1: ", station_3_pos1)
# print("possible position 2: ", station_3_pos2)

In [ ]:
%matplotlib inline
plt.figure()
plt.scatter([station_1_pos2[2][0], station_2_pos2[2][0], station_3_pos2[2][0]],[station_1_pos2[2][1], station_2_pos2[2][1], station_3_pos2[2][1]],c=['r', 'g', 'b'])

In [ ]:
%matplotlib inline

tfu = importlib.reload(tfu)
# choose which line is right for each total station in this section (the correct position is the first or the 2nd found with the resection)
# for i in range(len(station_1_pos1)):
#     Tf1 = tfu.tf_from_pose_roll_pitch_yaw(station_1_pos1[i])
#     Tf12 = ttfu.tf_from_pose_roll_pitch_yaw(station_1_pos2[i])

#     Tf2 = tfu.tf_from_pose_roll_pitch_yaw(station_2_pos1[i])
#     Tf22 = ttfu.tf_from_pose_roll_pitch_yaw(station_2_pos2[i])

#     Tf3 = tfu.tf_from_pose_roll_pitch_yaw(station_3_pos1[i])
#     Tf32 = ttfu.tf_from_pose_roll_pitch_yaw(station_3_pos2[i])
#     print(Tf1)
#     print(Tf12)
#     print(Tf2)
#     print(Tf22)
#     print(Tf3)
#     print(Tf32)
TF1 = []
TF2 = []
TF3 = []
tp1 = []
tp2 = []
tp3 = []
TR1 = []
TR2 = []
TR3 = []
# Calculate the transformation to realign all the total stations in the pillars' frame.
for i in range(len(station_1_pos2)):
# i = 2

    # we always use pos2 as it is always the correct possible pose between pos1 and pos2.
    TF1.append(tfu.tf_from_pose_roll_pitch_yaw(station_1_pos2[i]))
    TF2.append(tfu.tf_from_pose_roll_pitch_yaw(station_2_pos2[i]))
    TF3.append(tfu.tf_from_pose_roll_pitch_yaw(station_3_pos2[i]))
    #print(TF1)
    #print(TF2)
    #print(TF3)
    # print(trimble_1)
    # print(trimble_2)
    # print(trimble_3)
    tp1.append(TF1[i]@trimble_1)
    tp2.append(TF2[i]@trimble_2)
    tp3.append(TF3[i]@trimble_3)
    tp1[i] = tp1[i].T
    tp2[i] = tp2[i].T
    tp3[i] = tp3[i].T

    #print(tp1)
    #print(tp2)
    #print(tp3)

    TR1.append(calculate_tf_angle_using_scalar_product(tp1[i], liste_piliers, i))
    TF1[i] = TF1[i]@TR1[i]
    tp1[i] = TF1[i]@trimble_1
    tp1[i] = tp1[i].T

    TR2.append(calculate_tf_angle_using_scalar_product(tp2[i], liste_piliers, i))
    TF2[i] = TF2[i]@TR2[i]
    tp2[i] = TF2[i]@trimble_2
    tp2[i] = tp2[i].T

    TR3.append(calculate_tf_angle_using_scalar_product(tp3[i], liste_piliers, i))
    TF3[i] = TF3[i]@TR3[i]
    tp3[i] = TF3[i]@trimble_3
    tp3[i] = tp3[i].T

In [ ]:
%matplotlib inline

# This cell calculates the error between the measures from each total station for each pair of pillars

error_validation_points = []
error_control_points = []
counter = 0
for i,j,k in zip(tp1[0],tp2[0],tp3[0]):
        #print("i: ", i, "j: ", j, "k: ", k)
        #print("i: ", np.round(i, 2))
        #print("j: ", np.round(j, 2))
        #print("k: ", np.round(k, 2))
        dist_12 = np.linalg.norm(i-j)
        dist_13 = np.linalg.norm(i-k)
        dist_23 = np.linalg.norm(k-j)
        if counter < 2:
            error_control_points.append(dist_12)
            error_control_points.append(dist_13)
            error_control_points.append(dist_23)
        else:
            error_validation_points.append(dist_12)
            error_validation_points.append(dist_13)
            error_validation_points.append(dist_23)
        counter += 1

counter = 0
for i,j,k in zip(tp1[1],tp2[1],tp3[1]):
        #print("i: ", i, "j: ", j, "k: ", k)
        dist_12 = np.linalg.norm(i-j)
        dist_13 = np.linalg.norm(i-k)
        dist_23 = np.linalg.norm(k-j)
        if counter==0 or counter==2:
            error_control_points.append(dist_12)
            error_control_points.append(dist_13)
            error_control_points.append(dist_23)
        else:
            error_validation_points.append(dist_12)
            error_validation_points.append(dist_13)
            error_validation_points.append(dist_23)
        counter += 1
error_12_x = []
error_12_y = []
error_12_z = []
error_13_x = []
error_13_y = []
error_13_z = []
error_23_x = []
error_23_y = []
error_23_z = []
error_1_x = []
error_1_y = []
error_1_z = []
error_2_x = []
error_2_y = []
error_2_z = []
error_3_x = []
error_3_y = []
error_3_z = []
counter = 0
for i,j,k in zip(tp1[2],tp2[2],tp3[2]):
        #print("i: ", i, "j: ", j, "k: ", k)
        dist_12 = np.linalg.norm(i-j)
        dist_13 = np.linalg.norm(i-k)
        dist_23 = np.linalg.norm(k-j)
        if counter==0 or counter==3:
            error_control_points.append(dist_12)
            error_control_points.append(dist_13)
            error_control_points.append(dist_23)
        else:
            error_validation_points.append(dist_12)
            error_validation_points.append(dist_13)
            error_validation_points.append(dist_23)
        error_12_x.append(i[0]-j[0])
        error_12_y.append(i[1]-j[1])
        error_12_z.append(i[2]-j[2])
        error_13_x.append(i[0]-k[0])
        error_13_y.append(i[1]-k[1])
        error_13_z.append(i[2]-k[2])
        error_23_x.append(j[0]-k[0])
        error_23_y.append(j[1]-k[1])
        error_23_z.append(j[2]-k[2])
        error_1_x.append(i[0]-liste_piliers[counter][0])
        error_1_y.append(i[1]-liste_piliers[counter][1])
        error_1_z.append(i[2]-liste_piliers[counter][2])
        error_2_x.append(j[0]-liste_piliers[counter][0])
        error_2_y.append(j[1]-liste_piliers[counter][1])
        error_2_z.append(j[2]-liste_piliers[counter][2])
        error_3_x.append(k[0]-liste_piliers[counter][0])
        error_3_y.append(k[1]-liste_piliers[counter][1])
        error_3_z.append(k[2]-liste_piliers[counter][2])
        counter += 1

counter = 0
for i,j,k in zip(tp1[3],tp2[3],tp3[3]):
        #print("i: ", i, "j: ", j, "k: ", k)
        dist_12 = np.linalg.norm(i-j)
        dist_13 = np.linalg.norm(i-k)
        dist_23 = np.linalg.norm(k-j)
        if counter==1 or counter==2:
            error_control_points.append(dist_12)
            error_control_points.append(dist_13)
            error_control_points.append(dist_23)
        else:
            error_validation_points.append(dist_12)
            error_validation_points.append(dist_13)
            error_validation_points.append(dist_23)
        counter += 1

counter = 0
for i,j,k in zip(tp1[4],tp2[4],tp3[4]):
        #print("i: ", i, "j: ", j, "k: ", k)
        dist_12 = np.linalg.norm(i-j)
        dist_13 = np.linalg.norm(i-k)
        dist_23 = np.linalg.norm(k-j)
        if counter==1 or counter==3:
            error_control_points.append(dist_12)
            error_control_points.append(dist_13)
            error_control_points.append(dist_23)
        else:
            error_validation_points.append(dist_12)
            error_validation_points.append(dist_13)
            error_validation_points.append(dist_23)
        counter += 1

counter = 0
for i,j,k in zip(tp1[5],tp2[5],tp3[5]):
        #print("i: ", i, "j: ", j, "k: ", k)
        dist_12 = np.linalg.norm(i-j)
        dist_13 = np.linalg.norm(i-k)
        dist_23 = np.linalg.norm(k-j)
        if counter==2 or counter==3:
            error_control_points.append(dist_12)
            error_control_points.append(dist_13)
            error_control_points.append(dist_23)
        else:
            error_validation_points.append(dist_12)
            error_validation_points.append(dist_13)
            error_validation_points.append(dist_23)
        counter += 1

print(error_control_points)
print(error_validation_points)
print("Median error for the control points: ", round(np.median(error_control_points)*1000, 3), "mm")
print("Median error for the validation points: ", round(np.median(error_validation_points)*1000, 3), "mm")
for i in range(len(TF1)):
    tfu.plot_trajectories_prism(3, tp1[i].T, tp2[i].T, tp3[i].T, TF1[i], TF2[i], TF3[i], 0, 0,"example.pdf",0)

In [ ]:
%matplotlib inline

tfu = importlib.reload(tfu)
# choose which line is right for each total station in this section (the correct position is the first or the 2nd found with the resection)
# for i in range(len(station_1_pos1)):
#     Tf1 = tfu.tf_from_pose_roll_pitch_yaw(station_1_pos1[i])
#     Tf12 = ttfu.tf_from_pose_roll_pitch_yaw(station_1_pos2[i])

#     Tf2 = tfu.tf_from_pose_roll_pitch_yaw(station_2_pos1[i])
#     Tf22 = ttfu.tf_from_pose_roll_pitch_yaw(station_2_pos2[i])

#     Tf3 = tfu.tf_from_pose_roll_pitch_yaw(station_3_pos1[i])
#     Tf32 = ttfu.tf_from_pose_roll_pitch_yaw(station_3_pos2[i])
#     print(Tf1)
#     print(Tf12)
#     print(Tf2)
#     print(Tf22)
#     print(Tf3)
#     print(Tf32)
TF1 = []
TF2 = []
TF3 = []
tp1 = []
tp2 = []
tp3 = []
TR1 = []
TR2 = []
TR3 = []

# for i in range(len(station_1_pos2)):
i = 2
TF1.append(tfu.tf_from_pose_roll_pitch_yaw(station_1_pos2[i]))
TF2.append(tfu.tf_from_pose_roll_pitch_yaw(station_2_pos2[i]))
TF3.append(tfu.tf_from_pose_roll_pitch_yaw(station_3_pos2[i]))
#print(TF1)
#print(TF2)
#print(TF3)
#print(trimble_1)
#print(trimble_2)
#print(trimble_3)
tp1.append(TF1[0]@trimble_1)
tp2.append(TF2[0]@trimble_2)
tp3.append(TF3[0]@trimble_3)
tp1[0] = tp1[0].T
tp2[0] = tp2[0].T
tp3[0] = tp3[0].T

#print(tp1)
#print(tp2)
#print(tp3)

TR1.append(calculate_tf_angle_using_scalar_product(tp1[0], liste_piliers, i))
TF1[0] = TF1[0]@TR1[0]
tp1[0] = TF1[0]@trimble_1
tp1[0] = tp1[0].T

TR2.append(calculate_tf_angle_using_scalar_product(tp2[0], liste_piliers, i))
TF2[0] = TF2[0]@TR2[0]
tp2[0] = TF2[0]@trimble_2
tp2[0] = tp2[0].T

TR3.append(calculate_tf_angle_using_scalar_product(tp3[0], liste_piliers, i))
TF3[0] = TF3[0]@TR3[0]
tp3[0] = TF3[0]@trimble_3
tp3[0] = tp3[0].T

In [ ]:
%matplotlib notebook

error_validation_points = []
error_control_points = []
# counter = 0
# for i,j,k in zip(tp1[0],tp2[0],tp3[0]):
#         #print("i: ", i, "j: ", j, "k: ", k)
#         #print("i: ", np.round(i, 2))
#         #print("j: ", np.round(j, 2))
#         #print("k: ", np.round(k, 2))
#         dist_12 = np.linalg.norm(i-j)
#         dist_13 = np.linalg.norm(i-k)
#         dist_23 = np.linalg.norm(k-j)
#         if counter < 2:
#             error_control_points.append(dist_12)
#             error_control_points.append(dist_13)
#             error_control_points.append(dist_23)
#         else:
#             error_validation_points.append(dist_12)
#             error_validation_points.append(dist_13)
#             error_validation_points.append(dist_23)
#         counter += 1

# counter = 0
# for i,j,k in zip(tp1[1],tp2[1],tp3[1]):
#         #print("i: ", i, "j: ", j, "k: ", k)
#         dist_12 = np.linalg.norm(i-j)
#         dist_13 = np.linalg.norm(i-k)
#         dist_23 = np.linalg.norm(k-j)
#         if counter==0 or counter==2:
#             error_control_points.append(dist_12)
#             error_control_points.append(dist_13)
#             error_control_points.append(dist_23)
#         else:
#             error_validation_points.append(dist_12)
#             error_validation_points.append(dist_13)
#             error_validation_points.append(dist_23)
#         counter += 1
error_12_x = []
error_12_y = []
error_12_z = []
error_13_x = []
error_13_y = []
error_13_z = []
error_23_x = []
error_23_y = []
error_23_z = []
error_1_x = []
error_1_y = []
error_1_z = []
error_2_x = []
error_2_y = []
error_2_z = []
error_3_x = []
error_3_y = []
error_3_z = []
counter = 0
for i,j,k in zip(tp1[2],tp2[2],tp3[2]):
        #print("i: ", i, "j: ", j, "k: ", k)
        dist_12 = np.linalg.norm(i-j)
        dist_13 = np.linalg.norm(i-k)
        dist_23 = np.linalg.norm(k-j)
        if counter==0 or counter==3:
            error_control_points.append(dist_12)
            error_control_points.append(dist_13)
            error_control_points.append(dist_23)
        else:
            error_validation_points.append(dist_12)
            error_validation_points.append(dist_13)
            error_validation_points.append(dist_23)
        error_12_x.append(i[0]-j[0])
        error_12_y.append(i[1]-j[1])
        error_12_z.append(i[2]-j[2])
        error_13_x.append(i[0]-k[0])
        error_13_y.append(i[1]-k[1])
        error_13_z.append(i[2]-k[2])
        error_23_x.append(j[0]-k[0])
        error_23_y.append(j[1]-k[1])
        error_23_z.append(j[2]-k[2])
        error_1_x.append(i[0]-liste_piliers[counter][0])
        error_1_y.append(i[1]-liste_piliers[counter][1])
        error_1_z.append(i[2]-liste_piliers[counter][2])
        error_2_x.append(j[0]-liste_piliers[counter][0])
        error_2_y.append(j[1]-liste_piliers[counter][1])
        error_2_z.append(j[2]-liste_piliers[counter][2])
        error_3_x.append(k[0]-liste_piliers[counter][0])
        error_3_y.append(k[1]-liste_piliers[counter][1])
        error_3_z.append(k[2]-liste_piliers[counter][2])
        counter += 1

# counter = 0
# for i,j,k in zip(tp1[3],tp2[3],tp3[3]):
#         #print("i: ", i, "j: ", j, "k: ", k)
#         dist_12 = np.linalg.norm(i-j)
#         dist_13 = np.linalg.norm(i-k)
#         dist_23 = np.linalg.norm(k-j)
#         if counter==1 or counter==2:
#             error_control_points.append(dist_12)
#             error_control_points.append(dist_13)
#             error_control_points.append(dist_23)
#         else:
#             error_validation_points.append(dist_12)
#             error_validation_points.append(dist_13)
#             error_validation_points.append(dist_23)
#         counter += 1

# counter = 0
# for i,j,k in zip(tp1[4],tp2[4],tp3[4]):
#         #print("i: ", i, "j: ", j, "k: ", k)
#         dist_12 = np.linalg.norm(i-j)
#         dist_13 = np.linalg.norm(i-k)
#         dist_23 = np.linalg.norm(k-j)
#         if counter==1 or counter==3:
#             error_control_points.append(dist_12)
#             error_control_points.append(dist_13)
#             error_control_points.append(dist_23)
#         else:
#             error_validation_points.append(dist_12)
#             error_validation_points.append(dist_13)
#             error_validation_points.append(dist_23)
#         counter += 1

# counter = 0
# for i,j,k in zip(tp1[5],tp2[5],tp3[5]):
#         #print("i: ", i, "j: ", j, "k: ", k)
#         dist_12 = np.linalg.norm(i-j)
#         dist_13 = np.linalg.norm(i-k)
#         dist_23 = np.linalg.norm(k-j)
#         if counter==2 or counter==3:
#             error_control_points.append(dist_12)
#             error_control_points.append(dist_13)
#             error_control_points.append(dist_23)
#         else:
#             error_validation_points.append(dist_12)
#             error_validation_points.append(dist_13)
#             error_validation_points.append(dist_23)
#         counter += 1

print(error_control_points)
print(error_validation_points)
print("Median error for the control points: ", round(np.median(error_control_points)*1000, 3), "mm")
print("Median error for the validation points: ", round(np.median(error_validation_points)*1000, 3), "mm")
for i in range(len(TF1)):
    tfu.plot_trajectories_prism(3, tp1[i].T, tp2[i].T, tp3[i].T, TF1[i], TF2[i], TF3[i], 0, 0,"example.pdf",0)

In [ ]:
# Check error in distance for measurements
Distance = [np.linalg.norm(pilier_A-pilier_B),np.linalg.norm(pilier_A-pilier_C),np.linalg.norm(pilier_A-pilier_D),
           np.linalg.norm(pilier_B-pilier_C),np.linalg.norm(pilier_B-pilier_D),np.linalg.norm(pilier_C-pilier_D)]
error_1 = []
error_2 = []
error_3 = []
error_1.append(abs(np.linalg.norm(trimble_1[:,0]-trimble_1[:,1])-Distance[0])*1000)
error_1.append(abs(np.linalg.norm(trimble_1[:,0]-trimble_1[:,2])-Distance[1])*1000)
error_1.append(abs(np.linalg.norm(trimble_1[:,0]-trimble_1[:,3])-Distance[2])*1000)
error_1.append(abs(np.linalg.norm(trimble_1[:,1]-trimble_1[:,2])-Distance[3])*1000)
error_1.append(abs(np.linalg.norm(trimble_1[:,1]-trimble_1[:,3])-Distance[4])*1000)
error_1.append(abs(np.linalg.norm(trimble_1[:,2]-trimble_1[:,3])-Distance[5])*1000)

error_2.append(abs(np.linalg.norm(trimble_2[:,0]-trimble_2[:,1])-Distance[0])*1000)
error_2.append(abs(np.linalg.norm(trimble_2[:,0]-trimble_2[:,2])-Distance[1])*1000)
error_2.append(abs(np.linalg.norm(trimble_2[:,0]-trimble_2[:,3])-Distance[2])*1000)
error_2.append(abs(np.linalg.norm(trimble_2[:,1]-trimble_2[:,2])-Distance[3])*1000)
error_2.append(abs(np.linalg.norm(trimble_2[:,1]-trimble_2[:,3])-Distance[4])*1000)
error_2.append(abs(np.linalg.norm(trimble_2[:,2]-trimble_2[:,3])-Distance[5])*1000)

error_3.append(abs(np.linalg.norm(trimble_3[:,0]-trimble_3[:,1])-Distance[0])*1000)
error_3.append(abs(np.linalg.norm(trimble_3[:,0]-trimble_3[:,2])-Distance[1])*1000)
error_3.append(abs(np.linalg.norm(trimble_3[:,0]-trimble_3[:,3])-Distance[2])*1000)
error_3.append(abs(np.linalg.norm(trimble_3[:,1]-trimble_3[:,2])-Distance[3])*1000)
error_3.append(abs(np.linalg.norm(trimble_3[:,1]-trimble_3[:,3])-Distance[4])*1000)
error_3.append(abs(np.linalg.norm(trimble_3[:,2]-trimble_3[:,3])-Distance[5])*1000)
print(error_1)
print(error_2)
print(error_3)
print(np.mean(error_1),np.mean(error_2),np.mean(error_3))

In [ ]:
%matplotlib inline

pilier_A = np.array([0, 0, 0])
pilier_B = np.array([0, np.sqrt(181.016**2 - 0.323**2), 0.323])
pilier_C = np.array([0, np.sqrt(363.492**2 - 1.038**2), 1.038])
pilier_D = np.array([0, np.sqrt(548.578**2 - 2.374**2), 2.374])

#pilier_A = np.array([0, 0, 0])
#pilier_B = np.array([np.sqrt(181.016**2 - 0.323**2), 0, 0.323])
#pilier_C = np.array([np.sqrt(363.492**2 - 1.038**2), 0, 1.038])
#pilier_D = np.array([np.sqrt(548.578**2 - 2.374**2), 0, 2.374])

#pilier_A = np.array([0, 20, 0])
#pilier_B = np.array([0, 10, 0])
#pilier_C = np.array([0, -10, 0])
#pilier_D = np.array([0, -20, 0])

liste_piliers = [pilier_A, pilier_B, pilier_C, pilier_D]

# Read markers file of grand axe and show them
tfu = importlib.reload(tfu)
ttfu = importlib.reload(ttfu)

file_name = './data/total_station/20220715_prisms_calib/theodolite_reference_prisms_geomaic.txt'

raw_1, raw_2, raw_3, trimble_1, trimble_2, trimble_3, T_1_grand, T_2_grand, T_3_grand = ttfu.read_marker_file_raw_data(file_name)
station_1_pos1 = []
station_1_pos2 = []
station_2_pos1 = []
station_2_pos2 = []
station_3_pos1 = []
station_3_pos2 = []

for i in range(len(liste_piliers)):
    for j in range(i+1, len(liste_piliers)):
        print(i,j)
        possible_poses_station_1 = resection_with_2_known_points(liste_piliers[i], liste_piliers[j], trimble_1[0:3,i], trimble_1[0:3,j])
        for pose, station in zip(possible_poses_station_1, [station_1_pos1, station_1_pos2]):
            station.append(pose)
        possible_poses_station_2 = resection_with_2_known_points(liste_piliers[i], liste_piliers[j], trimble_2[0:3,i], trimble_2[0:3,j])
        for pose, station in zip(possible_poses_station_2, [station_2_pos1, station_2_pos2]):
            station.append(pose)
        possible_poses_station_3 = resection_with_2_known_points(liste_piliers[i], liste_piliers[j], trimble_3[0:3,i], trimble_3[0:3,j])
        for pose, station in zip(possible_poses_station_3, [station_3_pos1, station_3_pos2]):
            station.append(pose)

#station_1_pos1, station_1_pos2 = resection_with_2_known_points(pilier_D, pilier_C, raw_1[0], raw_1[1])
#station_2_pos1, station_2_pos2 = resection_with_2_known_points(pilier_D, pilier_C, raw_2[0], raw_2[1])
#station_3_pos1, station_3_pos2 = resection_with_2_known_points(pilier_D, pilier_C, raw_3[0], raw_3[1])
print("Sation 1:")
print("possible position 1: ", station_1_pos1)
print("possible position 2: ", station_1_pos2)
print("Station 2:")
print("possible position 1: ", station_2_pos1)
print("possible position 2: ", station_2_pos2)
print("Station 3:")
print("possible position 1: ", station_3_pos1)
print("possible position 2: ", station_3_pos2)

In [ ]:
%matplotlib inline
plt.figure()
plt.scatter([station_1_pos2[2][0], station_2_pos2[2][0], station_3_pos2[2][0]],[station_1_pos2[2][1], station_2_pos2[2][1], station_3_pos2[2][1]],c=['r', 'g', 'b'])

In [ ]:
%matplotlib inline

tfu = importlib.reload(tfu)
# choose which line is right for each total station in this section (the correct position is the first or the 2nd found with the resection)
# for i in range(len(station_1_pos1)):
#     Tf1 = tfu.tf_from_pose_roll_pitch_yaw(station_1_pos1[i])
#     Tf12 = ttfu.tf_from_pose_roll_pitch_yaw(station_1_pos2[i])

#     Tf2 = tfu.tf_from_pose_roll_pitch_yaw(station_2_pos1[i])
#     Tf22 = ttfu.tf_from_pose_roll_pitch_yaw(station_2_pos2[i])

#     Tf3 = tfu.tf_from_pose_roll_pitch_yaw(station_3_pos1[i])
#     Tf32 = ttfu.tf_from_pose_roll_pitch_yaw(station_3_pos2[i])
#     print(Tf1)
#     print(Tf12)
#     print(Tf2)
#     print(Tf22)
#     print(Tf3)
#     print(Tf32)
TF1 = []
TF2 = []
TF3 = []
tp1 = []
tp2 = []
tp3 = []
TR1 = []
TR2 = []
TR3 = []

for i in range(len(station_1_pos2)):
# i = 2
    TF1.append(tfu.tf_from_pose_roll_pitch_yaw(station_1_pos2[i]))
    TF2.append(tfu.tf_from_pose_roll_pitch_yaw(station_2_pos2[i]))
    TF3.append(tfu.tf_from_pose_roll_pitch_yaw(station_3_pos2[i]))
    #print(TF1)
    #print(TF2)
    #print(TF3)
    print(trimble_1)
    print(trimble_2)
    print(trimble_3)
    tp1.append(TF1[i]@trimble_1)
    tp2.append(TF2[i]@trimble_2)
    tp3.append(TF3[i]@trimble_3)
    tp1[i] = tp1[i].T
    tp2[i] = tp2[i].T
    tp3[i] = tp3[i].T

    #print(tp1)
    #print(tp2)
    #print(tp3)

    TR1.append(calculate_tf_angle_using_scalar_product(tp1[i], liste_piliers, i))
    TF1[i] = TF1[i]@TR1[i]
    tp1[i] = TF1[i]@trimble_1
    tp1[i] = tp1[i].T

    TR2.append(calculate_tf_angle_using_scalar_product(tp2[i], liste_piliers, i))
    TF2[i] = TF2[i]@TR2[i]
    tp2[i] = TF2[i]@trimble_2
    tp2[i] = tp2[i].T

    TR3.append(calculate_tf_angle_using_scalar_product(tp3[i], liste_piliers, i))
    TF3[i] = TF3[i]@TR3[i]
    tp3[i] = TF3[i]@trimble_3
    tp3[i] = tp3[i].T

In [ ]:
%matplotlib inline

error_validation_points = []
error_control_points = []
counter = 0
for i,j,k in zip(tp1[0],tp2[0],tp3[0]):
        #print("i: ", i, "j: ", j, "k: ", k)
        #print("i: ", np.round(i, 2))
        #print("j: ", np.round(j, 2))
        #print("k: ", np.round(k, 2))
        dist_12 = np.linalg.norm(i-j)
        dist_13 = np.linalg.norm(i-k)
        dist_23 = np.linalg.norm(k-j)
        if counter < 2:
            error_control_points.append(dist_12)
            error_control_points.append(dist_13)
            error_control_points.append(dist_23)
        else:
            error_validation_points.append(dist_12)
            error_validation_points.append(dist_13)
            error_validation_points.append(dist_23)
        counter += 1

counter = 0
for i,j,k in zip(tp1[1],tp2[1],tp3[1]):
        #print("i: ", i, "j: ", j, "k: ", k)
        dist_12 = np.linalg.norm(i-j)
        dist_13 = np.linalg.norm(i-k)
        dist_23 = np.linalg.norm(k-j)
        if counter==0 or counter==2:
            error_control_points.append(dist_12)
            error_control_points.append(dist_13)
            error_control_points.append(dist_23)
        else:
            error_validation_points.append(dist_12)
            error_validation_points.append(dist_13)
            error_validation_points.append(dist_23)
        counter += 1
error_12_x = []
error_12_y = []
error_12_z = []
error_13_x = []
error_13_y = []
error_13_z = []
error_23_x = []
error_23_y = []
error_23_z = []
error_1_x = []
error_1_y = []
error_1_z = []
error_2_x = []
error_2_y = []
error_2_z = []
error_3_x = []
error_3_y = []
error_3_z = []
counter = 0
for i,j,k in zip(tp1[2],tp2[2],tp3[2]):
        #print("i: ", i, "j: ", j, "k: ", k)
        dist_12 = np.linalg.norm(i-j)
        dist_13 = np.linalg.norm(i-k)
        dist_23 = np.linalg.norm(k-j)
        if counter==0 or counter==3:
            error_control_points.append(dist_12)
            error_control_points.append(dist_13)
            error_control_points.append(dist_23)
        else:
            error_validation_points.append(dist_12)
            error_validation_points.append(dist_13)
            error_validation_points.append(dist_23)
        error_12_x.append(i[0]-j[0])
        error_12_y.append(i[1]-j[1])
        error_12_z.append(i[2]-j[2])
        error_13_x.append(i[0]-k[0])
        error_13_y.append(i[1]-k[1])
        error_13_z.append(i[2]-k[2])
        error_23_x.append(j[0]-k[0])
        error_23_y.append(j[1]-k[1])
        error_23_z.append(j[2]-k[2])
        error_1_x.append(i[0]-liste_piliers[counter][0])
        error_1_y.append(i[1]-liste_piliers[counter][1])
        error_1_z.append(i[2]-liste_piliers[counter][2])
        error_2_x.append(j[0]-liste_piliers[counter][0])
        error_2_y.append(j[1]-liste_piliers[counter][1])
        error_2_z.append(j[2]-liste_piliers[counter][2])
        error_3_x.append(k[0]-liste_piliers[counter][0])
        error_3_y.append(k[1]-liste_piliers[counter][1])
        error_3_z.append(k[2]-liste_piliers[counter][2])
        counter += 1

counter = 0
for i,j,k in zip(tp1[3],tp2[3],tp3[3]):
        #print("i: ", i, "j: ", j, "k: ", k)
        dist_12 = np.linalg.norm(i-j)
        dist_13 = np.linalg.norm(i-k)
        dist_23 = np.linalg.norm(k-j)
        if counter==1 or counter==2:
            error_control_points.append(dist_12)
            error_control_points.append(dist_13)
            error_control_points.append(dist_23)
        else:
            error_validation_points.append(dist_12)
            error_validation_points.append(dist_13)
            error_validation_points.append(dist_23)
        counter += 1

counter = 0
for i,j,k in zip(tp1[4],tp2[4],tp3[4]):
        #print("i: ", i, "j: ", j, "k: ", k)
        dist_12 = np.linalg.norm(i-j)
        dist_13 = np.linalg.norm(i-k)
        dist_23 = np.linalg.norm(k-j)
        if counter==1 or counter==3:
            error_control_points.append(dist_12)
            error_control_points.append(dist_13)
            error_control_points.append(dist_23)
        else:
            error_validation_points.append(dist_12)
            error_validation_points.append(dist_13)
            error_validation_points.append(dist_23)
        counter += 1

counter = 0
for i,j,k in zip(tp1[5],tp2[5],tp3[5]):
        #print("i: ", i, "j: ", j, "k: ", k)
        dist_12 = np.linalg.norm(i-j)
        dist_13 = np.linalg.norm(i-k)
        dist_23 = np.linalg.norm(k-j)
        if counter==2 or counter==3:
            error_control_points.append(dist_12)
            error_control_points.append(dist_13)
            error_control_points.append(dist_23)
        else:
            error_validation_points.append(dist_12)
            error_validation_points.append(dist_13)
            error_validation_points.append(dist_23)
        counter += 1

print(error_control_points)
print(error_validation_points)
print("Median error for the control points: ", round(np.median(error_control_points)*1000, 3), "mm")
print("Median error for the validation points: ", round(np.median(error_validation_points)*1000, 3), "mm")
for i in range(len(TF1)):
    tfu.plot_trajectories_prism(3, tp1[i].T, tp2[i].T, tp3[i].T, TF1[i], TF2[i], TF3[i], 0, 0,"example.pdf",0)

In [ ]:
print(error_12_x)
print(error_12_y)
print(error_12_z)
print(error_13_x)
print(error_13_y)
print(error_13_z)
print(error_23_x)
print(error_23_y)
print(error_23_z)
trimble_1 = trimble_1.T
print(np.linalg.norm(tp1[0][0]-tp1[0][3])-548.578)
print(np.linalg.norm(trimble_1[0] - trimble_1[3])-548.578)
print(np.linalg.norm(liste_piliers[0]-liste_piliers[3])-548.578)
print(error_1_x)
print(error_1_y)
print(error_1_z)
print(error_2_x)
print(error_2_y)
print(error_2_z)
print(error_3_x)
print(error_3_y)
print(error_3_z)

In [ ]:
fig = plt.figure(figsize =(16, 6))
ax = fig.add_subplot(111)

box = ax.boxplot([error_control_points, error_validation_points], notch=True, patch_artist=True, vert = 1, showfliers=False, showmeans=False)
plt.xticks([1, 2], ["Control points \n\n Median: "+str(round(np.median(error_control_points)*1000,2))+"mm \n Std: "+str(round(np.std(error_control_points)*1000,2))+"mm",
                          "Validation points \n\n Median: "+str(round(np.median(error_validation_points)*1000,2))+"mm \n Std: "+str(round(np.std(error_validation_points)*1000,2))+"mm"])
colors_box = ['#069AF3', '#EF4026']

for patch, color in zip(box['boxes'], colors_box):
    patch.set_facecolor(color)
        
ax.set_ylabel("Resection error [m]")
plt.show()
#fig.savefig("./figs/new_resection_datasets_prior_only-f-2-1-1-1-6-1-L.jpg")

In [ ]:
print(error_12_x)
print(error_12_y)
print(error_12_z)
print(error_13_x)
print(error_13_y)
print(error_13_z)
print(error_23_x)
print(error_23_y)
print(error_23_z)
trimble_1 = trimble_1.T
print(np.linalg.norm(tp1[0][0]-tp1[0][3])-548.578)
print(np.linalg.norm(trimble_1[0] - trimble_1[3])-548.578)
print(np.linalg.norm(liste_piliers[0]-liste_piliers[3])-548.578)
print(error_1_x)
print(error_1_y)
print(error_1_z)
print(error_2_x)
print(error_2_y)
print(error_2_z)
print(error_3_x)
print(error_3_y)
print(error_3_z)

In [ ]:
fig = plt.figure(figsize =(16, 6))
ax = fig.add_subplot(111)

box = ax.boxplot([error_control_points, error_validation_points], notch=True, patch_artist=True, vert = 1, showfliers=False, showmeans=False)
plt.xticks([1, 2], ["Control points \n\n Median: "+str(round(np.median(error_control_points)*1000,2))+"mm \n Std: "+str(round(np.std(error_control_points)*1000,2))+"mm",
                          "Validation points \n\n Median: "+str(round(np.median(error_validation_points)*1000,2))+"mm \n Std: "+str(round(np.std(error_validation_points)*1000,2))+"mm"])
colors_box = ['#069AF3', '#EF4026']

for patch, color in zip(box['boxes'], colors_box):
    patch.set_facecolor(color)
        
ax.set_ylabel("Resection error [m]")
plt.show()
#fig.savefig("./figs/new_resection_datasets_prior_only-f-2-1-1-1-6-1-L.jpg")

In [ ]:
for i in range(len(station_1_pos2)):
    print(station_1_pos2[i] - station_2_pos2[i])
    print(station_1_pos2[i] - station_3_pos2[i])
    print(station_2_pos2[i] - station_3_pos2[i])

In [ ]:
# print('station 1, distance A-D après TF', (np.linalg.norm(tp1[0][0]-tp1[0][3])-548.578)*1000, 'mm')
print('station 1, distance A-D avant TF', (np.linalg.norm(trimble_1[:,0] - trimble_1[:,3])-548.578)*1000, 'mm')
# print('station 1, distance A-C après TF', (np.linalg.norm(tp1[0][0]-tp1[0][2])-363.492)*1000, 'mm')
print('station 1, distance A-C avant TF', (np.linalg.norm(trimble_1[:,0] - trimble_1[:,2])-363.492)*1000, 'mm')
# print('station 1, distance A-B après TF', (np.linalg.norm(tp1[0][0]-tp1[0][1])-181.016)*1000, 'mm')
print('station 1, distance A-B avant TF', (np.linalg.norm(trimble_1[:,0] - trimble_1[:,1])-181.016)*1000, 'mm')
# print('station 2, distance A-D après TF', (np.linalg.norm(tp2[0][0]-tp2[0][3])-548.578)*1000, 'mm')
print('station 2, distance A-D avant TF', (np.linalg.norm(trimble_2[:,0] - trimble_2[:,3])-548.578)*1000, 'mm')
# print('station 2, distance A-C après TF', (np.linalg.norm(tp2[0][0]-tp2[0][2])-363.492)*1000, 'mm')
print('station 2, distance A-C avant TF', (np.linalg.norm(trimble_2[:,0] - trimble_2[:,2])-363.492)*1000, 'mm')
# print('station 2, distance A-B après TF', (np.linalg.norm(tp2[0][0]-tp2[0][1])-181.016)*1000, 'mm')
print('station 2, distance A-B avant TF', (np.linalg.norm(trimble_2[:,0] - trimble_2[:,1])-181.016)*1000, 'mm')
# print('station 3, distance A-D après TF', (np.linalg.norm(tp3[0][0]-tp3[0][3])-548.578)*1000, 'mm')
print('station 3, distance A-D avant TF', (np.linalg.norm(trimble_3[:,0] - trimble_3[:,3])-548.578)*1000, 'mm')
# print('station 3, distance A-C après TF', (np.linalg.norm(tp3[0][0]-tp3[0][2])-363.492)*1000, 'mm')
print('station 3, distance A-C avant TF', (np.linalg.norm(trimble_3[:,0] - trimble_3[:,2])-363.492)*1000, 'mm')
# print('station 3, distance A-B après TF', (np.linalg.norm(tp3[0][0]-tp3[0][1])-181.016)*1000, 'mm')
print('station 3, distance A-B avant TF', (np.linalg.norm(trimble_3[:,0] - trimble_3[:,1])-181.016)*1000, 'mm')